In [5]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, InputLayer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [6]:
dataset_crop_weeds = 'agri_data/data/'  # Adjust this path to your dataset

# Image Sizes for MobileNetV2
img_height = 224
img_width = 224

# Defines batch size and number of epochs
batch_size = 32
epochs = 10


In [7]:
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (img_width, img_height))
    return image / 255.0

def parse_annotation(annotation_path):
    with open(annotation_path, 'r') as file:
        class_label = int(file.readline().strip().split()[0])
    return class_label


In [8]:
images = []
annotations = []

for filename in os.listdir(dataset_crop_weeds):
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_crop_weeds, filename)
        images.append(image_path)
    elif filename.endswith('.txt'):
        annotation_path = os.path.join(dataset_crop_weeds, filename)
        annotations.append(annotation_path)

# Split the data
train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.2, random_state=42)
train_images, val_images, train_annotations, val_annotations = train_test_split(train_images, train_annotations, test_size=0.25, random_state=42)

train_images_data = np.array([load_and_preprocess_image(img) for img in train_images])
val_images_data = np.array([load_and_preprocess_image(img) for img in val_images])
test_images_data = np.array([load_and_preprocess_image(img) for img in test_images])

train_annotations_data = np.array([parse_annotation(ann) for ann in train_annotations])
val_annotations_data = np.array([parse_annotation(ann) for ann in val_annotations])
test_annotations_data = np.array([parse_annotation(ann) for ann in test_annotations])


In [9]:
data_gen_args = dict(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
train_datagen = ImageDataGenerator(**data_gen_args)


In [10]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False


In [11]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [14]:
history = model.fit(train_datagen.flow(train_images_data, train_annotations_data, batch_size=batch_size), epochs=epochs, validation_data=(val_images_data, val_annotations_data), callbacks=[early_stopping])


Epoch 1/10
25/25 [==============================] - 9s 319ms/step - loss: 0.8477 - accuracy: 0.4923 - val_loss: 0.7133 - val_accuracy: 0.4731
Epoch 2/10
25/25 [==============================] - 7s 274ms/step - loss: 0.7675 - accuracy: 0.5321 - val_loss: 0.7134 - val_accuracy: 0.5038
Epoch 3/10
25/25 [==============================] - 7s 273ms/step - loss: 0.7726 - accuracy: 0.5167 - val_loss: 0.7227 - val_accuracy: 0.5038
Epoch 4/10
25/25 [==============================] - 6s 247ms/step - loss: 0.7344 - accuracy: 0.5256 - val_loss: 0.7264 - val_accuracy: 0.4808
Epoch 5/10
25/25 [==============================] - 6s 243ms/step - loss: 0.7207 - accuracy: 0.5436 - val_loss: 0.7528 - val_accuracy: 0.4962
Epoch 6/10
25/25 [==============================] - 6s 245ms/step - loss: 0.7695 - accuracy: 0.4936 - val_loss: 0.7374 - val_accuracy: 0.4962
Epoch 7/10
25/25 [==============================] - 6s 246ms/step - loss: 0.7312 - accuracy: 0.5538 - val_loss: 0.7578 - val_accuracy: 0.5154
Epoch 

In [15]:
test_loss, test_acc = model.evaluate(test_images_data, test_annotations_data, verbose=2)
print(f"Test accuracy: {test_acc}")


9/9 - 1s - loss: 0.7463 - accuracy: 0.4962 - 1s/epoch - 167ms/step
Test accuracy: 0.4961538314819336
